# TP 11 - Benchmark et complexité

Pour chaque exercice, rentrez votre réponse dans l'éditeur Python associé.
Enregistrez vos modifications, de préférence au format ipynb, lorsque vous aurez terminé.

## Exercice 1 - Génération de nombres aléatoires

Il est possible d'utiliser la fonction `randint` pour générer un entier aléatoire entre 2 bornes. Exécutez plusieurs fois le code ci-dessous et observez que les résultats diffèrent :

In [10]:
import random

random.randint(1, 6)

2

Le code ci-dessus permet de simuler le comportement d'un dé à 6 faces, puisqu'il renvoie des valeurs entre 1 et 6.

En pratique, les nombres renvoyés sont semi-aléatoires. En effet, l'implémentation des fonctions de génération de nombres aléatoires repose sur des séries mathématiques prévisibles. Il est possible d'initialiser la série avec un nombre appelé la graine (*seed* en anglais). La seed peut être initialisée par exemple avec l'adresse MAC d'une carte réseau, ou encore l'heure actuelle, afin d'améliorer le caractère aléatoire de la série.

Pour ce TP, vous n'aurez pas besoin d'initialiser la seed.

Ecrivez une fonction `genere_liste_aleatoire` qui prend 3 arguments :
* N : le nombre de nombres aléatoires à générer,
* min : la plus petite valeur à générer (-1 000 000 par défaut),
* max : la plus grande valeur à générer (1 000 000 par défaut).

et renvoie une liste de `N` nombres aléatoires entre `min` et `max`.

Par exemple :
```py
L = genere_liste_aleatoire(2)
print(L)
sixDesQuatreFaces = genere_liste_aleatoire(6, min=1, max=4)
print(sixDesQuatreFaces)
```

peut afficher par exemple :
```
[1234, -6789]
[4, 1, 3, 1, 2, 4]
```

## Exercice 2 - Benchmarking

Nous avons vu qu'une manière simple d'instrumenter le code pour obtenir un temps d'exécution approximatif consiste à faire la différence entre le temps avant et après le code à mesurer :

In [15]:
from datetime import datetime

debut = datetime.now()

genere_liste_aleatoire(1000000)

fin = datetime.now()
temps_ecoule = fin - debut
print(f"Temps d'exécution : {temps_ecoule.total_seconds()}s")

Temps d'exécution : 0.731853s


Il s'agit d'un résultat peu précis car il dépend des autres processus tournant sur le système d'exploitation. On peut obtenir un résultat équivalent avec `time.process_time` :

In [16]:
import time

debut = time.process_time()

genere_liste_aleatoire(1000000)

fin = time.process_time()
temps_ecoule = fin - debut
print(f"Temps d'exécution : {temps_ecoule}s")

Temps d'exécution : 0.7083241349999998s


Ce résultat n'est pas beaucoup plus précis, même s'il se cantonne à mesurer le temps d'exécution du processus courant, plutôt que le temps global de la machine.

La bibliothèque `timeit` permet d'obtenir un résultat beaucoup plus précis, car indépendant des perturbations du système d'exploitation :

In [17]:
from timeit import timeit

test = lambda: genere_liste_aleatoire(1000000)

timeit(test, number=1)

0.6955388720002702

Le premier argument de `timeit` doit être une fonction qui ne prend pas d'argument. C'est la raison pour laquelle on construit la fonction `test` avec une lambda qui appelle `genere_liste_aleatoire` avec le nombre que l'on souhaite.

Le deuxième argument de `timeit` est le nombre de fois qu'il doit exécuter la fonction en argument. Par exemple, l'appel suivant prendra 10 fois plus de temps, puisqu'on appelle 10 fois la fonction `genere_liste_aleatoire` :

In [18]:
timeit(test, number=10)

6.591189424999811

Il est possible d'aller plus loin en employant un profiler. Exécutez le code suivant :

In [19]:
from cProfile import Profile
from pstats import Stats

profiler = Profile()
profiler.runcall(test)

stats = Stats(profiler)
stats.strip_dirs()
stats.sort_stats('cumulative')
stats.print_stats()

         5048757 function calls in 1.346 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    1.346    1.346 4194484203.py:3(<lambda>)
        1    0.000    0.000    1.346    1.346 3811730195.py:1(genere_liste_aleatoire)
        1    0.200    0.200    1.346    1.346 3811730195.py:2(<listcomp>)
  1000000    0.239    0.000    1.146    0.000 random.py:334(randint)
  1000000    0.455    0.000    0.906    0.000 random.py:290(randrange)
  1000000    0.322    0.000    0.452    0.000 random.py:237(_randbelow_with_getrandbits)
  1048753    0.073    0.000    0.073    0.000 {method 'getrandbits' of '_random.Random' objects}
  1000000    0.056    0.000    0.056    0.000 {method 'bit_length' of 'int' objects}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




Le résultat est une table d'information organisée par fonction. Cette table fournit des informations sur le temps d'exécution de notre lambda (qui s'appelle `test`), et de toutes les fonctions qu'elle appelle de manière directe et indirecte.

Ainsi, notre lambda appelle la fonction `genere_liste_aléatoire`, qui appelle `randint`, qui appelle `randrange`, qui appelle des fonctions internes de la bibliothèque standard Python.

Sur chaque ligne, on obtient les informations suivantes :
* **nbcalls** : nombre de fois que la fonction est appelée.
* **tottime** : nombre de secondes passées à exécuter la fonction, en excluant le temps passé dans les autres fonctions que celle-ci appelle.
* **tottime percall** : nombre moyen de secondes passées à exécuter la fonction, en excluant le temps passé dans les autres fonctions que celle-ci appelle. Autrement dit : $\frac{tottime}{nbcalls}$.
* **cumtime** : temps cumulatif passé à exécuter la fonction, en prenant compte aussi le temps passé dans les fonctions appelées.
* **cumtime percall** : nombre moyen de secondes passées à exécuter la fonction, en prenant compte aussi le temps passé dans les fonctions appelées. Autrement dit : $\frac{cumtime}{nbcalls}$.

## Exercice 3 - Recherche linéaire versus recherche dichotomique

Dans les exercices 1 et 2 du TP 5, vous avez implémenté une recherche linéaire et une recherche dichotomique.

Vous n'aviez pas encore appris le concept de fonction.

On peut réécrire ce code sous forme de fonctions. Les fonctions `recherche_lineaire` et `recherche_dichotomique`. Ces 2 fonctions prennent 2 paramètres :
* une liste,
* un élément potentiellement dans cette liste.

Ces 2 fonctions retournent -1 si l'élément n'est pas trouvé. Si l'élément est trouvé, l'index de cet élément dans la liste est renvoyé.

In [3]:
def recherche_lineaire(L, element):
    """Recherche element dans L et retourne son index ou -1."""
    for i in range(len(L)):
        if L[i] == element:
            return i
    
    return -1

def recherche_dichotomique(L, element):
    """Recherche element dans L triée et retourne son index ou -1."""
    debut = 0
    fin = len(L) - 1

    while debut <= fin:
        milieu = debut + (fin - debut) // 2
        actuel = L[milieu]

        if element < actuel:
            fin = milieu - 1
        elif element > actuel:
            debut = milieu + 1
        else:
            return milieu

    return -1
    
L = [1, 2, 3, 4, 5]
index_lineaire = recherche_lineaire(L, 5)
print(f"index : {index_lineaire}")
index_dichoto = recherche_dichotomique(L, 5)
print(f"index : {index_dichoto}")
index_dichoto = recherche_dichotomique(L, 1)
print(f"index : {index_dichoto}")
index_dichoto = recherche_dichotomique(L, 4)
print(f"index : {index_dichoto}")
index_dichoto = recherche_dichotomique(L, -3)
print(f"index : {index_dichoto}")
index_dichoto = recherche_dichotomique(L, 10)
print(f"index : {index_dichoto}")
index_dichoto = recherche_dichotomique(L, 2.5)
print(f"index : {index_dichoto}")

index : 4
index : 4
index : 0
index : 3
index : -1
index : -1
index : -1



Utilisez `genere_liste_aleatoire` pour générer des listes aléatoires comportant :
* 10 éléments,
* 1 000 éléments,
* 100 000 éléments,
* 1 000 000 éléments,
* 10 000 000 éléments.

Nommez respectivement ces listes `L1`, `L3`, `L5`, `L6` et `L7`. Utilisez pour cela des variables.

Utilisez la méthode `sort` pour garantir que ces listes sont triées.

Par exemple :
```py
L1 = genere_liste_aleatoire(10**1).sort()
```

On cherche à mesurer le pire cas. Dans les variables `dernier_L1`, `dernier_L3`, `dernier_L5`, `dernier_L6` et `dernier_L7`, mettez respectivement le dernier élément des listes `L1`, `L3`, `L5`, `L6` et `L7`.

Créez les lambdas :
* `test_lineaire_L1` qui appelle `recherche_lineaire` avec `L1` et `dernier_L1`.
* `test_dicho_L1` qui appelle `recherche_dichotomique` avec `L1` et `dernier_L1`.

De manière similaire, créez les lambdas suivantes :
* `test_lineaire_L3` et `test_dicho_L3`,
* `test_lineaire_L5` et `test_dicho_L5`,
* `test_lineaire_L6` et `test_dicho_L6`,
* `test_lineaire_L7` et `test_dicho_L7`,

Utilisez `timeit` pour mesurer ces différentes lambda. Prenez un papier et un crayon et tracez les courbes obtenues.

Quelle est la complexité des fonctions `recherche_lineaire` et `recherche_dichotomique` ? Que déduisez-vous de vos résultats ?

## Exercice 4 - Bulles

Que fait la fonction `bulle` dans le code suivant ?

In [ ]:
def echange(a, b):
    return b, a

def bulle(liste):
    N = len(liste)
    for _ in range(N):
        for i in range(N):
            if liste[i] > liste[i + 1] :
                liste[i], liste[i + 1] = echange(liste[i], liste[i + 1])

Quelle est la complexité de la fonction `bulle` ?

Quel serait le pire cas pour cette fonction ? On entend par là le cas où il y aurait le plus d'opérations à effectuer dans cet algorithme.

En utilisant une approche similaire à celle de l'exercice 3, utilisez `timeit` pour mesurer le temps d'exécution pour 10, 1000, 100 000, 1 000 000 et 10 000 000 d'éléments.

Utilisez `Profiler` et `Stats` pour mesurer le nombre d'appels à la fonction `echange`.

Une `list` comporte une méthode `sort`. Il est donc possible d'écrire :
```py
L = [5, 1, 3, 2, 4]
L.sort()
```

Effectuez les mêmes tests, en utilisant `timeit`, puis `Profiler` et `Stats`, pour comparer la fonction `bulle` avec la méthode `sort`. Prenez un papier et un crayon et tracez les courbes.

Qu'en déduisez-vous ?

## Exercice 5 - Groupe de puissance

On souhaite, pour une liste `L`, générer une liste de listes qui contient toutes les combinaisons possibles des éléments de `L`. On appelle cette liste de listes le groupe de puissance.

Par exemple, si `L` est `["x", "y"]`, alors le groupe de puissance est la liste contenant les listes `[]`, `["x"]`, `["y"]` et `["x", "y"]`.

Autre exemple, si `L` est `[1, 2, 3]`, alors le groupe de puissance est : `[[], [1], [2], [3], [1, 2], [1, 3], [2, 3], [1, 2, 3]]`

Il est possible de représenter toutes ces combinaisons par une chaîne de `N` caractères `0` et de `1`, où `1` représente la présence de l'élément, et `0` son absence. La combinaison représentant aucun élément est représentée par une chaîne ne comportant que des `0`. La combinaison représentant la présence de tous les éléments est représentée par la chaîne ne comportant que des `1`. La combinaison contenant uniquement le 1er et le dernier élément est représentée par `100...001`.

Par exemple, si `L` est `["x", "y", "z"]`, alors `b = "101"` génère la liste `["x", "z"]`.



### Exercice 5.1 - Conversion d'un nombre en base 2

Ecrivez une fonction `representation_binaire` qui prend un nombre `N` et un nombre de digits `nb_digits` qui renvoie la représentation binaire de `N` ayant au minimum `nb_digits` caractères.

Par exemple :
```py
b = representation_binaire(5, nb_digits=8)
print(b)
```

doit afficher :
```
00000101
```

Pensez à utiliser les opérateurs modulo (`%`) et division entière (`//`) pour convertir le nombre `N` en base 2.

Pensez également à rajouter, si nécessaire, des 0 au début de la chaîne résultante.

### Exercice 5.2 - Génération du groupe de puissance

Ecrivez une fonction `groupe_puissance` qui prend en entrée une liste `L` et renvoie la liste des listes qui contient toutes les combinaisons possibles d'éléments de `L`.

Votre fonction va comporter une première boucle allant de 0 à $2^{len(L)}$. A chaque itération, vous allez utiliser `representation_binaire` pour générer la ième représentation binaire de taille `len(L)`.

Une boucle imbriquée va ensuite parser la chaîne de caractères en retour de `representation_binaire`. A chaque fois qu'un `"1"` est trouvé, on ajoute dans la sous-liste courante le jème élément de `L` (j étant l'index de la boucle imbriquée).

### Exercice 5.3 - Benchmark

Quelle est la complexité de `groupe_puissance` ?

En utilisant `timeit`, mesurez le temps d'exécution des cas d'usage suivant de `groupe_puissance` :
* L2 = `["x", "y"]`
* L3 = `["x", "y", "z"]`
* L5 = `["a", "b", "c", "d", "e"]`
* L10 = `list(range(10))`
* L15 = `list(range(15))`

Effectuez les mêmes tests avec `Profile` et `Stats`, notamment pour voir concrètement le nombre d'appels à `representation_binaire`.

Comme précédemment, prenez un papier et un crayon, et tracez les résultats que vous avez obtenu.
Qu'en déduisez-vous ?